In [3]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [4]:
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Weaviate
from typing import List
import os
import weaviate

def openai_chat_client():
  deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_ID")
  key = os.getenv("AZURE_OPENAI_API_KEY")
  resource = os.getenv("AZURE_OPENAI_RESOURCE_NAME")

  return AzureChatOpenAI(deployment_name=deployment, 
                         openai_api_key=key, 
                         openai_api_base=f"https://{resource}.openai.azure.com/",
                         openai_api_version="2023-03-15-preview")


def weaviate_vector_store(index_name: str, text_key: str, attributes: List[str] = []):
  weaviate_url = os.environ['WEAVIATE_URL']
  weaviate_api_key = os.environ['WEAVIATE_API_KEY']
  openai_api_key = os.environ['AZURE_OPENAI_API_KEY']

  auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

  client = weaviate.Client(
      url=weaviate_url,
      auth_client_secret=auth_config,
      additional_headers={
          "X-OpenAI-Api-Key": openai_api_key
      }
  )
  return Weaviate(client=client, 
                  index_name=index_name, 
                  text_key=text_key, 
                  attributes=attributes)

In [5]:
import base64
import json
import os
from langchain.chains import RetrievalQAWithSourcesChain


from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)


def ask(question):
  index_name = "Work"
  text_key = "title"
  attributes = ["identifier", "title", "source", "alternate_title", "contributor", "create_date", "creator", "date_created", "description", "genre", "keywords", "language", "location", "physical_description_material", "physical_description_size", "scope_and_contents", "style_period", "subject", "table_of_contents", "technique"]

  weaviate = weaviate_vector_store(
    index_name=index_name, 
    text_key=text_key, 
    attributes=attributes
  )
  
  client = openai_chat_client()
  chain = RetrievalQAWithSourcesChain.from_chain_type(
    client, 
    chain_type="stuff", 
    retriever=weaviate.as_retriever()
  )

  return chain({"question": question})

In [6]:
jprint(ask("Where is Northwestern University located?"))

<IPython.core.display.JSON object>

In [7]:
jprint(ask("Is Northwestern near Lake Michigan?"))

<IPython.core.display.JSON object>

In [11]:
jprint(ask("Does the collection have any images of lions in the wild?"))

<IPython.core.display.JSON object>

In [12]:
jprint(ask("Can you show me examples of war propaganda?"))

<IPython.core.display.JSON object>

In [15]:
jprint(ask("Can you show me historical Northwestern football matches?"))

<IPython.core.display.JSON object>

In [18]:
jprint(ask("Who did John Cage correspond with during his life?"))

<IPython.core.display.JSON object>